<a href="https://colab.research.google.com/github/componavt/LLLE-R1900s/blob/main/src/visualization/loans_map_timeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📈 Rural Credit Trends in Early 20th Century Russia

This notebook visualizes loan amount trends over time, broken down by:
- **Settlements** (villages, volosts, etc.)
- **Credit types** (e.g., FarmBuildings, Livestock, Housing)

Data source: cleaned CSV from `csv_out/`, specified in `config.env`.

Built for Google Colab. Just click "Run All" to generate plots.

In [1]:
# Install required packages (if not already installed)
!pip install -q python-dotenv pandas plotly

In [2]:
# Clone the repo if running in Colab (optional — if data not uploaded manually)
import os

if not os.path.exists('LLLE-R1900s'):
    !git clone https://github.com/componavt/LLLE-R1900s.git
    %cd LLLE-R1900s
else:
    %cd LLLE-R1900s

Cloning into 'LLLE-R1900s'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 135 (delta 71), reused 92 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (135/135), 593.58 KiB | 3.03 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/LLLE-R1900s


In [4]:
import os
import pandas as pd
from dotenv import load_dotenv
import plotly.express as px
import plotly.graph_objects as go

# Load configuration
load_dotenv('config.env')

# Get paths
csv_out_dir = os.getenv('CSV_OUT_DIR', 'data/csv_out')
society_file = 'data/society_settlement.csv'
output_file_name = os.getenv('OUTPUT_CSV_FILE')

if not output_file_name:
    csv_files = [f for f in os.listdir(csv_out_dir) if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError("No CSV files found in the output directory.")
    output_file_name = csv_files[0]

csv_path = os.path.join(csv_out_dir, output_file_name)
print(f"Loading loan data from: {csv_path}")

# Load loan data
df_loans = pd.read_csv(csv_path)
print(f"Loaded {len(df_loans)} loan records.")

# Load society/settlement data with coordinates
df_society = pd.read_csv(society_file)
print(f"Loaded {len(df_society)} settlements with coordinates.")

# Merge loan data with coordinates by settlement name
df_merged = df_loans.merge(
    df_society[['English', 'Settlement', 'Latitude', 'Longitude']],
    left_on='settlement',
    right_on='English',
    how='left'
)

# Check for missing coordinates
missing = df_merged[df_merged['Latitude'].isna()]
if not missing.empty:
    print("⚠️ Warning: Some settlements have no coordinates:")
    print(missing['settlement'].unique())

# Drop rows without coordinates
df_merged = df_merged.dropna(subset=['Latitude', 'Longitude'])

# Create vertical offset for "3D effect" — shift latitude slightly by year
# Normalize year to create small vertical shifts (e.g., 0.01° per year)
min_year = df_merged['year'].min()
df_merged['Latitude_Offset'] = df_merged['Latitude'] + (df_merged['year'] - min_year) * 0.015

# Optional: Define a consistent color map for credit items
credit_items = df_merged['credit_item'].unique()
color_map = {item: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]
             for i, item in enumerate(credit_items)}

# Create the plot
fig = go.Figure()

for credit_type in credit_items:
    df_subset = df_merged[df_merged['credit_item'] == credit_type]
    fig.add_trace(go.Scattermapbox(
        lat=df_subset['Latitude_Offset'],
        lon=df_subset['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=df_subset['amount_rubles'] / df_merged['amount_rubles'].max() * 50 + 5,  # scale size
            color=color_map[credit_type],
            opacity=0.8
        ),
        name=credit_type,
        hovertemplate=(
            "<b>Settlement</b>: %{customdata[0]}<br>" +
            "<b>Year</b>: %{customdata[1]}<br>" +
            "<b>Credit Type</b>: %{customdata[2]}<br>" +
            "<b>Amount</b>: %{customdata[3]:.2f} руб.<br>" +
            "<b>Loans</b>: %{customdata[4]}<extra></extra>"
        ),
        customdata=df_subset[['Settlement', 'year', 'credit_item', 'amount_rubles', 'loan_count']].values
    ))

# Set map layout
fig.update_layout(
    title="Loans by Settlement, Year, and Credit Type (Vertical Timeline)",
    mapbox=dict(
        style="open-street-map",
        center=dict(lat=df_merged['Latitude'].mean(), lon=df_merged['Longitude'].mean()),
        zoom=6
    ),
    margin={"r":0,"t":50,"l":0,"b":0},
    legend_title="Credit Type",
    height=800
)

# Ensure outputs directory exists
os.makedirs('outputs', exist_ok=True)

# Save and show
fig.write_html("outputs/loans_map_3d_timeline.html")
fig.show()

print("✅ Interactive map saved to: outputs/loans_map_3d_timeline.html")

Loading loan data from: data/csv_out/loans_s4_i16.csv
Loaded 148 loan records.
Loaded 29 settlements with coordinates.


✅ Interactive map saved to: outputs/loans_map_3d_timeline.html
